In [7]:
#pip install Tashaphyne


In [8]:
#pip install emoji

In [9]:
pip install transformers

In [10]:
pip install torchmetrics

In [11]:
pip install pytorch_lightning

In [12]:
#pip install emoji

## importing libaraires


In [14]:

# Add environment Packages paths to conda
import os, sys, warnings
import pandas as pd
import numpy as np
warnings.simplefilter("ignore")

# Text preprocessing packages
import nltk # Text libarary
# nltk.download('stopwords')
import string # Removing special characters {#, @, ...}

# Modelling
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.svm import SVC
# Saving Model
import pickle

# Visualization Packages
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.3)
%matplotlib inline


from pylab import rcParams
import joblib

from transformers import AutoTokenizer, AutoModel
import torch
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics.functional import f1
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, classification_report

from tqdm.auto import tqdm

torch.cuda.empty_cache()


In [15]:
torch.cuda.empty_cache()

In [16]:
#import the dataset
path_input = '../input/cleaned-data-iris/cleaned_data_IRIS.csv'
df = pd.read_csv(path_input)
df.dropna(inplace=True)
df.head()

In [17]:
#print the shape of the dataset
df.shape

In [18]:
#check for null values
df.isnull().sum()

In [19]:
#check for duplication
df.duplicated().sum()

In [20]:
#make train and test data and save it as a csv file
train, val = train_test_split(df[['label','text']], test_size=0.2)


lbl_enc = LabelEncoder()
train.loc[:,"label"] = lbl_enc.fit_transform(train["label"])
val.loc[:,"label"] = lbl_enc.transform(val["label"])

joblib.dump(lbl_enc,"label_encoder.pkl")

train.to_csv("train.csv",index=False)
val.to_csv("test.csv",index=False)

In [21]:
#print the classes and it's encoding
lbl_enc.classes_
{v: k for v, k in enumerate(lbl_enc.classes_)}

In [22]:
#make a new data to test different examples in our data
new_data = df.iloc[:2]
new_data

new_text  = {'text': 'حب','label': 'love'}
new_data = new_data.append(new_text, ignore_index = True)
new_text  = {'text': 'مستغرب','label': 'surprise'}
new_data = new_data.append(new_text, ignore_index = True)
new_text  = {'text': 'حزن','label': 'sadness'}
new_data = new_data.append(new_text, ignore_index = True)
new_data.loc[:,"label"] = lbl_enc.transform(new_data["label"])
new_data.to_csv("new_data.csv",index=False)

new_data


In [23]:
class ArabicDataset(Dataset):
    def __init__(self,data,max_len,model_type="Twitter"):
        super().__init__()
        self.labels = data["label"].values
        self.texts = data["text"].values
        self.max_len = max_len
        model = {"Twitter": "aubmindlab/bert-base-arabertv02-twitter",
                "ARBERT": "UBC-NLP/ARBERT",
                "Base": "aubmindlab/bert-base-arabertv02",
                "MARBERT": "UBC-NLP/MARBERT"}
        self.tokenizer = AutoTokenizer.from_pretrained(model[model_type])
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self,idx):
        text = " ".join(self.texts[idx].split())
        label = self.labels[idx]
        inputs = self.tokenizer(text,padding='max_length',
                                max_length=self.max_len,truncation=True,return_tensors="pt")
        
        #input_ids,token_type_ids,attention_mask
        return {
            "inputs":{"input_ids":inputs["input_ids"][0],
                      "token_type_ids":inputs["token_type_ids"][0],
                      "attention_mask":inputs["attention_mask"][0],
                     },
            "labels": torch.tensor(label,dtype=torch.long) 
        }

In [24]:
class ArabicDataModule(pl.LightningDataModule):
    def __init__(self,train_path,val_path,new_data_path,batch_size=12,max_len=100,model_type="Twitter"):
        super().__init__()
        self.train_path,self.val_path,self.new_data_path= train_path,val_path,new_data_path
        self.batch_size = batch_size
        self.max_len = max_len
        self.model_type = model_type
    
    def setup(self,stage=None):
        train = pd.read_csv(self.train_path)
        val = pd.read_csv(self.val_path)
        new_data = pd.read_csv(self.new_data_path)
        self.train_dataset = ArabicDataset(data=train,max_len=self.max_len,model_type=self.model_type)
        self.val_dataset = ArabicDataset(data=val,max_len=self.max_len,model_type=self.model_type)
        self.new_dataset = ArabicDataset(data=new_data,max_len=self.max_len,model_type=self.model_type)

    def train_dataloader(self):
        return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.new_dataset,batch_size=self.batch_size,shuffle=False)
    
    def test_dataloader(self):
        return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False)

In [25]:
n_classes = 8
class ArabicBertModel(pl.LightningModule):
    def __init__(self,model_type="Twitter"):
        super().__init__()
        model = {"Twitter": ("aubmindlab/bert-base-arabertv02-twitter",768),
                "ARBERT": ("UBC-NLP/ARBERT",768),
                 "Base": ("aubmindlab/bert-base-arabertv02",768),
                "MARBERT": ("UBC-NLP/MARBERT",768)}
        self.bert_model = AutoModel.from_pretrained(model[model_type][0])
        self.fc = nn.Linear(model[model_type][1],n_classes)
    
    def forward(self,inputs):
        out = self.bert_model(**inputs)#inputs["input_ids"],inputs["token_type_ids"],inputs["attention_mask"])
        last_hidden_states = out[1]
        out = self.fc(last_hidden_states)
        return out
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=0.0001)
    
    def criterion(self,output,target):
        return nn.CrossEntropyLoss()(output,target)
    
    #TODO: adding metrics
    def training_step(self,batch,batch_idx):
        x,y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_result = f1(out, y, num_classes=n_classes)
        metrics = {"train_f1": f1_result, "train_loss": loss}
        self.log_dict(metrics)
        return loss
    
    def validation_step(self,batch,batch_idx):
        x, y = batch["inputs"],batch["labels"]
        out = self(x)
        loss = self.criterion(out,y)
        f1_result = f1(out, y, num_classes=n_classes)
        metrics = {"val_f1": f1_result, "val_loss": loss}
        self.log_dict(metrics)
        return metrics

## Training on Arabert Twitter v02

In [26]:
# TODO: getting different models sizes results
MODEL_TYPE = "Twitter"
dm = ArabicDataModule(train_path="./train.csv",
                val_path = "./test.csv",
                new_data_path = "./new_data.csv",
                batch_size=128, max_len=70, model_type=MODEL_TYPE)

model = ArabicBertModel(model_type=MODEL_TYPE)
trainer = pl.Trainer(gpus=-1,max_epochs=1, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")]) 
trainer.fit(model,dm)

## Training Results

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

preds = []
real_values = []

test_dataloader = dm.test_dataloader()

progress_bar = tqdm(range(len(test_dataloader)))

model.eval()
for batch in test_dataloader:    
    x,y = batch["inputs"],batch["labels"]
    inp = {k: v.to(device) for k, v in x.items()}
    
    with torch.no_grad():
        outputs = model(inp)

    predictions = torch.argmax(outputs, dim=1)
    
    preds.extend(predictions)
    real_values.extend(y)

    progress_bar.update()
    
preds = torch.stack(preds).cpu()
real_values = torch.stack(real_values).cpu()
print(classification_report(real_values, preds, target_names=lbl_enc.classes_))



## Save Model

In [32]:
torch.save(model,os.path.join('/', MODEL_TYPE))


## Load model

In [43]:
model = torch.load(os.path.join('/', MODEL_TYPE))

In [44]:
model.eval()


In [45]:
model.state_dict()